## Englarge the dataset

Data size for finetuning: 5000:
- MS Marco: 1/3 -> 1666
- Adversial QA: 1/3 -> 1666
- Squad 2.0: 1/3 -> 1666


For each set of QA, we can try two types of data preparing:
1. A Q123
2. A Q1, A Q2, A Q3

### Prepare dataset

##### Training (finetuning)

In [118]:
# squad 2.0
import pandas as pd
squad = pd.read_json('train-v2.0.json')
tmp = 0
qa_list = []
for i in range(len(squad['data'])):
    if len(qa_list) > 4000:
        break
    for j in range(len(squad['data'][i]['paragraphs'])):
        tmp = 0
        context = squad['data'][i]['paragraphs'][j]['context']
        for k in range(len(squad['data'][i]['paragraphs'][j]['qas'])):
            if squad['data'][i]['paragraphs'][j]['qas'][k]['answers']:
                if tmp == 2:
                    break
                answer = squad['data'][i]['paragraphs'][j]['qas'][k]['answers'][0]['text']
                question = squad['data'][i]['paragraphs'][j]['qas'][k]['question']
                qa_list.append([context, answer, question])
                tmp += 1


In [119]:
# MS MARCO
import gzip
import json
dataset_path = '/home/tzujohsu/SI630/dev_v1.1.json.gz'
ms_list = []
with gzip.open(dataset_path, "r") as f:
    for doc in f:
        if len(nq_list) == 5000:
            break
        doc = doc.decode('utf-8')
        doc = json.loads(doc)
        if doc['answers']:
            if 'what' in doc['query'] or 'how' in doc['query'] or 'why' in doc['query'] or 'which' in doc['query'] or 'when' in doc['query'] or 'where' in doc['query']:
                for i in doc['passages']:
                    if i['is_selected']:
                        context = (i['passage_text'])
                
                lst = ([len(i.split()) for i in doc['answers']])
                idx = lst.index(max(lst))
                answer = doc['answers'][idx]
                question = doc['query']
                ms_list.append([context, answer, question])
        

In [120]:
# aqa dev
import pandas as pd
aqa = pd.read_json('aqa-dev.json')
aqa = (list(aqa['data']))

aqa_list = []
for i in range(len(aqa)):
    if len(aqa_list) > 5000:
        break
    for j in range(len(aqa[i]['paragraphs'])):
        context = aqa[i]['paragraphs'][j]['context']
        for k in range(7):
            try:
                question = aqa[i]['paragraphs'][j]['qas'][k]['question']
                answer = aqa[i]['paragraphs'][j]['qas'][k]['answers'][0]['text']
                aqa_list.append([context, question, answer])
            except:
                pass



In [121]:
import random
random.shuffle(qa_list)
random.shuffle(ms_list)
random.shuffle(aqa_list)

In [122]:
combined = []
combined.extend(qa_list[:1667])
combined.extend(ms_list[:1667])
combined.extend(aqa_list[:1666])
len(combined)

5000

##### Evaluation

In [123]:
# 1500
eval = []
eval.extend(qa_list[1667:1667+500])
eval.extend(ms_list[1667:1667+500])
eval.extend(aqa_list[1667:1667+500])
len(eval)

1500

##### save data

In [1]:
import pickle
from tqdm import tqdm
# with open("combined.pickle", "wb") as fp:
#     pickle.dump(combined, fp)

# with open("eval.pickle", "wb") as fp:
#     pickle.dump(eval, fp)

with open("combined.pickle", "rb") as fp:
    combined = pickle.load(fp)

with open("eval.pickle", "rb") as fp:
    eval = pickle.load(fp)

### paraphrase

In [12]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bart_model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase').to(device)
bart_tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
t5_tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws", use_fast=False)
t5_model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
from sentence_transformers import SentenceTransformer

def cosine_similarity(tensor1, tensor2):
    """
    Compute the cosine similarity between two tensors.
    """
    tensor1_norm = tensor1 / tensor1.norm(dim=1, keepdim=True)
    tensor2_norm = tensor2 / tensor2.norm(dim=1, keepdim=True)
    return torch.mm(tensor1_norm, tensor2_norm.transpose(0, 1))

embedding_model = 'all-MiniLM-L6-v2'  # or any other model you prefer
sim_embedder = SentenceTransformer(embedding_model)

In [ ]:
enlarged_v1 = []
for data in tqdm(combined, total=5000):
    context, answer, question = data[0], data[1], data[2]
    paraphrases = []
    
    batch = bart_tokenizer(question, return_tensors='pt')
    generated_ids = bart_model.generate(batch['input_ids'], do_sample = True, max_new_tokens = 20,
                                   top_k = 250, top_p = 1.5, num_return_sequences = 3)

    paraphrases.extend(bart_tokenizer.batch_decode(generated_ids, skip_special_tokens=True))
    

    encoding = t5_tokenizer.encode_plus(question, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    generated_ids = t5_model.generate(input_ids=input_ids, attention_mask=attention_masks, do_sample = True, max_new_tokens = 50, 
                                    top_k = 250, top_p = 1.5, num_return_sequences = 3)
    for output in generated_ids:
        line = t5_tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        paraphrases.append(line)

    original_embedding = sim_embedder.encode([question], convert_to_tensor=True)
    paraphrase_embeddings = sim_embedder.encode(paraphrases, convert_to_tensor=True)

    # filter out paraphrased questions that have high similarities
    similarities = cosine_similarity(original_embedding, paraphrase_embeddings).squeeze(0).tolist()
    # selected_paraphrases = {paraphrase: similarity for paraphrase, similarity in zip(paraphrases, similarities) if 0.96 <= similarity <= 0.99}
    selected_paraphrases = [paraphrase for paraphrase, similarity in zip(paraphrases, similarities) if 0.96 <= similarity <= 0.99]
    if selected_paraphrases:
        q = [question]
        q.extend(selected_paraphrases)    
        enlarged_v1.append([context, answer, q])
    else:
        enlarged_v1.append([context, answer, [question]])
    

In [ ]:
enlarged_eval = []
for data in tqdm(eval, total=1500):
    context, answer, question = data[0], data[1], data[2]
    paraphrases = []
    
    batch = bart_tokenizer(question, return_tensors='pt')
    generated_ids = bart_model.generate(batch['input_ids'], do_sample = True, max_new_tokens = 20,
                                   top_k = 250, top_p = 1.5, num_return_sequences = 3)

    paraphrases.extend(bart_tokenizer.batch_decode(generated_ids, skip_special_tokens=True))
    

    encoding = t5_tokenizer.encode_plus(question, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    generated_ids = t5_model.generate(input_ids=input_ids, attention_mask=attention_masks, do_sample = True, max_new_tokens = 50, 
                                    top_k = 250, top_p = 1.5, num_return_sequences = 3)
    for output in generated_ids:
        line = t5_tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        paraphrases.append(line)

    original_embedding = sim_embedder.encode([question], convert_to_tensor=True)
    paraphrase_embeddings = sim_embedder.encode(paraphrases, convert_to_tensor=True)

    # filter out paraphrased questions that have high similarities
    similarities = cosine_similarity(original_embedding, paraphrase_embeddings).squeeze(0).tolist()
    # selected_paraphrases = {paraphrase: similarity for paraphrase, similarity in zip(paraphrases, similarities) if 0.96 <= similarity <= 0.99}
    selected_paraphrases = [paraphrase for paraphrase, similarity in zip(paraphrases, similarities) if 0.96 <= similarity <= 0.99]
    if selected_paraphrases:
        q = [question]
        q.extend(selected_paraphrases)    
        enlarged_eval.append([context, answer, q])
    else:
        enlarged_eval.append([context, answer, [question]])
    

In [ ]:
with open("ennlarged_finetune.pickle", "wb") as fp:
    pickle.dump(enlarged_v1, fp)

with open("ennlarged_eval.pickle", "wb") as fp:
    pickle.dump(enlarged_eval, fp)